# Benchmark

## Загрузка

In [1]:
!pip install langchain pymupdf transformers torch langchain_community rank_bm25 ragatouille sentence-transformers faiss-gpu accelerate bitsandbytes unstructured "unstructured[pdf]" update poppler-utils

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.3/990.3 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 22.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 MB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 35.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.2/374.2 kB 36.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 46.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Новый

In [1]:
import torch
from transformers import AutoModel, AutoTokenizer
import faiss
import numpy as np
import pandas as pd
from typing import List, Tuple, Dict, Any
import time
from IPython.display import Markdown, display



In [2]:
# Загрузка
model_ckpt = "intfloat/multilingual-e5-large" #"deepvk/USER-bge-m3"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model = AutoModel.from_pretrained(model_ckpt)

device = torch.device("cuda")
model.to(device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/418 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/690 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

XLMRobertaModel(
  (embeddings): XLMRobertaEmbeddings(
    (word_embeddings): Embedding(250002, 1024, padding_idx=1)
    (position_embeddings): Embedding(514, 1024, padding_idx=1)
    (token_type_embeddings): Embedding(1, 1024)
    (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): XLMRobertaEncoder(
    (layer): ModuleList(
      (0-23): 24 x XLMRobertaLayer(
        (attention): XLMRobertaAttention(
          (self): XLMRobertaSelfAttention(
            (query): Linear(in_features=1024, out_features=1024, bias=True)
            (key): Linear(in_features=1024, out_features=1024, bias=True)
            (value): Linear(in_features=1024, out_features=1024, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): XLMRobertaSelfOutput(
            (dense): Linear(in_features=1024, out_features=1024, bias=True)
            (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwi

In [3]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_text_splitters import CharacterTextSplitter


In [4]:
text_splitter = RecursiveCharacterTextSplitter( #20
    separators=["\n\n", "\n", " ", ""
    ],
    chunk_size=800,
    chunk_overlap=200
)

In [5]:
# Загрузка документов
from langchain.document_loaders import PyMuPDFLoader, DirectoryLoader
loader = DirectoryLoader("/content/drive/MyDrive/Legal_doc_NEW", glob="**/*.pdf", show_progress=True, loader_cls=PyMuPDFLoader)#.load_and_split()
documents = loader.load_and_split(text_splitter)
#documents=loader
texts = [doc.page_content for doc in documents]
metadata = [doc.metadata for doc in documents]
docs_df = pd.DataFrame({'text': texts, 'metadata': metadata})

100%|██████████| 32/32 [00:25<00:00,  1.24it/s]


In [ ]:
texts[0]

'Зарегистрировано в Минюсте России 2 июня 2023 г. N 73701\n\xa0\nФЕДЕРАЛЬНАЯ СЛУЖБА БЕЗОПАСНОСТИ РОССИЙСКОЙ ФЕДЕРАЦИИ\n\xa0\nПРИКАЗ\nот 11 мая 2023 г. N 213\n\xa0\nОБ УТВЕРЖДЕНИИ ПОРЯДКА\nОСУЩЕСТВЛЕНИЯ МОНИТОРИНГА ЗАЩИЩЕННОСТИ ИНФОРМАЦИОННЫХ\nРЕСУРСОВ, ПРИНАДЛЕЖАЩИХ ФЕДЕРАЛЬНЫМ ОРГАНАМ ИСПОЛНИТЕЛЬНОЙ\nВЛАСТИ, ВЫСШИМ ИСПОЛНИТЕЛЬНЫМ ОРГАНАМ ГОСУДАРСТВЕННОЙ ВЛАСТИ\nСУБЪЕКТОВ РОССИЙСКОЙ ФЕДЕРАЦИИ, ГОСУДАРСТВЕННЫМ ФОНДАМ,\nГОСУДАРСТВЕННЫМ КОРПОРАЦИЯМ (КОМПАНИЯМ), ИНЫМ ОРГАНИЗАЦИЯМ,\nСОЗДАННЫМ НА ОСНОВАНИИ ФЕДЕРАЛЬНЫХ ЗАКОНОВ, СТРАТЕГИЧЕСКИМ\nПРЕДПРИЯТИЯМ, СТРАТЕГИЧЕСКИМ АКЦИОНЕРНЫМ ОБЩЕСТВАМ\nИ СИСТЕМООБРАЗУЮЩИМ ОРГАНИЗАЦИЯМ РОССИЙСКОЙ ЭКОНОМИКИ,\nЮРИДИЧЕСКИМ ЛИЦАМ, ЯВЛЯЮЩИМСЯ СУБЪЕКТАМИ КРИТИЧЕСКОЙ\nИНФОРМАЦИОННОЙ ИНФРАСТРУКТУРЫ РОССИЙСКОЙ ФЕДЕРАЦИИ\nЛИБО ИСПОЛЬЗУЕМЫХ ИМИ'

In [6]:
def cls_pooling(model_output):
    return model_output.last_hidden_state[:, 0]

def get_embeddings(text_list: List[str], batch_size: int = 0):
    embeddings = []
    for i in range(0, len(text_list)):
        batch = text_list[i]
        encoded_input = tokenizer(
            batch, padding=True, truncation=True, return_tensors="pt"
        )
        encoded_input = {k: v.to(device) for k, v in encoded_input.items()}
        with torch.no_grad():
            model_output = model(**encoded_input)
        batch_embeddings = cls_pooling(model_output).detach().cpu().numpy()
        embeddings.append(batch_embeddings)
        torch.cuda.empty_cache()  # Освобождаем память GPU после обработки каждого батча
    return np.vstack(embeddings)

## Получение эмбеддингов для документов
embeddings = get_embeddings(texts, batch_size=0)
#
#import numpy as np
#import pickle
#
## Сохранение эмбеддингов в файл на Google Диск
#def save_embeddings_to_drive(embeddings, file_path):
#    with open(file_path, 'wb') as f:
#        pickle.dump(embeddings, f)
#

#embeddings_file_path = '/content/drive/MyDrive/embeddings.npy'
#save_embeddings_to_drive(embeddings, embeddings_file_path)


In [ ]:
import pickle
import numpy as np
embeddings_file_path = '/content/drive/MyDrive/embeddings.npy'
def load_embeddings_from_drive(file_path):
    with open(file_path, 'rb') as f:
        embeddings = pickle.load(f)
    return embeddings

# Пример загрузки
embeddings = load_embeddings_from_drive(embeddings_file_path)


In [7]:
# Создание индекса Faiss
faiss_index = faiss.IndexFlatL2(embeddings.shape[1])
faiss_index.add(embeddings)

faq_data = pd.read_excel('/content/drive/MyDrive/filtered_updated_faq_data (2).xlsx')

In [8]:
import pandas as pd
import time
from typing import List, Tuple
import faiss
import markdown
from IPython.display import display, Markdown

In [9]:
qwe=[]
def perform_combined_search(
    question: str,
    embeddings: List[List[float]],
    faiss_index: faiss.Index,
    num_retrieved_docs: int = 5,
    rewrite_question_flag: bool = False,
) -> Tuple[List[Tuple[str, float, str]], float]:
    start_time = time.time()


    qwe.append(question)
    question_embedding = get_embeddings([question])
    D, I = faiss_index.search(question_embedding, num_retrieved_docs)
    faiss_results = [(docs_df.iloc[i], D[0][j]) for j, i in enumerate(I[0])]
    combined_results = {}
    for doc, score in faiss_results:
        file_path = doc['metadata']['file_path']
        text_fragment = doc['text']
        combined_results[file_path] = (combined_results.get(file_path, (0, ""))[0] + score, text_fragment)
    combined_results = sorted(combined_results.items(), key=lambda x: x[1][0], reverse=True)
    retrieval_time = time.time() - start_time

    return [(res[0].split("/content/drive/MyDrive/Legal_doc_NEW/")[-1], res[1][0], res[1][1]) for res in combined_results], retrieval_time



results = []

for _, row in faq_data.iterrows():
    question = row['Question']
    expected_file_paths = row['PDF Title'].split(', ')
    unique_file_paths_with_scores, retrieval_time = perform_combined_search(question, embeddings, faiss_index)

    true_positives = len(set([res[0] for res in unique_file_paths_with_scores]) & set(expected_file_paths))
    false_positives = len(set([res[0] for res in unique_file_paths_with_scores]) - set(expected_file_paths))
    false_negatives = len(set(expected_file_paths) - set([res[0] for res in unique_file_paths_with_scores]))

    precision = true_positives / (true_positives + false_positives) if (true_positives + false_positives) > 0 else 0
    recall = true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) > 0 else 0

    results.append({
        'Question': question,
        'Expected File Paths': expected_file_paths,
        'Predicted File Paths with Scores': unique_file_paths_with_scores,
        'True Positives': true_positives,
        'False Positives': false_positives,
        'False Negatives': false_negatives,
        'Precision': precision,
        'Recall': recall,
        'Retrieval Time (s)': retrieval_time,
        'Rerank Time (s)': 0
    })

results_df = pd.DataFrame(results)

total_true_positives = results_df['True Positives'].sum()
total_false_positives = results_df['False Positives'].sum()
total_false_negatives = results_df['False Negatives'].sum()
total_predictions = total_true_positives + total_false_positives

accuracy = total_true_positives / total_predictions if total_predictions != 0 else 0
average_precision = results_df['Precision'].mean()
average_recall = results_df['Recall'].mean()
import markdown

print("-------------------------------------------------------------------")
print(f"Total True Positives: {total_true_positives}")
print(f"Total False Positives: {total_false_positives}")
print(f"Accuracy: {accuracy:.2f}")
print(f"Average Precision: {average_precision:.2f}")
print(f"Average Recall: {average_recall:.2f}")
print("-------------------------------------------------------------------")

for index, row in results_df.iterrows():
    display(Markdown(f"#Вопрос: {row['Question']}"))
    display(Markdown(f"Если есть llm: {qwe[index]}"))
    display(Markdown(f"##**Нужные PDF:**"))
    for path in row['Expected File Paths']:
        display(Markdown(f" - **{path}**"))
    print()
    display(Markdown("##**Предсказанные PDF:**"))
    for path, score, text_fragment in row['Predicted File Paths with Scores']:
        display(Markdown(f"  - **{path} (Score: {score:.2f})**"))
        display(Markdown(f"**Фрагмент текста:**"))
        #print(f"{text_fragment}")
        print()
    print(f"Время поиска (с): {row['Retrieval Time (s)']:.2f}")
    print(f"Время ранжирования (с): {row['Rerank Time (s)']:.2f}")
    print("")
#41 017

-------------------------------------------------------------------
Total True Positives: 44
Total False Positives: 281
Accuracy: 0.14
Average Precision: 0.19
Average Recall: 0.29
-------------------------------------------------------------------


#Вопрос: New!СБ ЗОКИИ: какие ограничения существуют по допуску иностранного гражданина к эксплуатации или обслуживанию значимого объекта КИИ?

Если есть llm: New!СБ ЗОКИИ: какие ограничения существуют по допуску иностранного гражданина к эксплуатации или обслуживанию значимого объекта КИИ?

##**Нужные PDF:**

 - **18) Приказ Минэнерго РФ от 13.12.2011 № 587 _Об утверждении перечня работ**

...

  - **1) Методический документ _Методика оценки показателя состояния технической защиты информации и обеспеч.pdf (Score: 298.57)**

**Фрагмент текста:**

  - **22) Приказ ФСТЭК России от 11.02.2013 № 17.pdf (Score: 295.14)**

**Фрагмент текста:**


Время поиска (с): 0.02
Время ранжирования (с): 0.00



In [10]:
# Создание индекса Faiss
faiss_index = faiss.IndexFlatL2(embeddings.shape[1])
faiss_index.add(embeddings)

faq_data = pd.read_excel('/content/drive/MyDrive/filtered_updated_faq_data (2).xlsx')

In [11]:
import re
# Function to extract numbers before ')'
def extract_numbers(text):
    numbers = re.findall(r'(\d+)\)', text)
    filtered_numbers = [num for num in numbers if len(num) <= 2]
    return ','.join(filtered_numbers)

# Apply function to 'PDF Title' column
faq_data['PDF Title'] = faq_data['PDF Title'].apply(extract_numbers)


In [12]:

# Function to clean the sets
def clean_sets(sets):
    cleaned_sets = []
    for s in sets:
        new_set = set()
        for item in s:
            if ',' in item:
                new_set.update(filter(None, item.split(',')))  # Split and filter out empty strings
            elif item != ',':
                new_set.add(item)
        cleaned_sets.append(new_set)
    return cleaned_sets


In [13]:
import pandas as pd
import time
from typing import List, Tuple
import faiss
import markdown
from IPython.display import display, Markdown

# Assuming qwe, embeddings, faiss_index, docs_df, faq_data, and clean_sets are already defined

def perform_combined_search(
    question: str,
    embeddings: List[List[float]],
    faiss_index: faiss.Index,
    num_retrieved_docs: int = 5,
    rewrite_question_flag: bool = False,
) -> Tuple[List[Tuple[str, float, str]], float]:
    start_time = time.time()

    qwe.append(question)
    question_embedding = get_embeddings([question])
    D, I = faiss_index.search(question_embedding, num_retrieved_docs)
    faiss_results = [(docs_df.iloc[i], D[0][j]) for j, i in enumerate(I[0])]
    combined_results = {}
    for doc, score in faiss_results:
        file_path = doc['metadata']['file_path']
        text_fragment = doc['text']

        combined_results[file_path] = (combined_results.get(file_path, (0, ""))[0] + score, text_fragment)

    combined_results = sorted(combined_results.items(), key=lambda x: x[1][0], reverse=True)
    retrieval_time = time.time() - start_time
    return [(extract_numbers(res[0]), res[1][0], res[1][1]) for res in combined_results], retrieval_time

results = []

for _, row in faq_data.iterrows():
    question = row['Question']
    expected_file_paths = set(row['PDF Title'].split(','))

    unique_file_paths_with_scores, retrieval_time = perform_combined_search(question, embeddings, faiss_index)
    unique_file_paths = set([res[0] for res in unique_file_paths_with_scores])

    # Clean the sets
    cleaned_expected_file_paths = clean_sets([expected_file_paths])[0]
    cleaned_unique_file_paths = clean_sets([unique_file_paths])[0]

    true_positives = len(cleaned_unique_file_paths & cleaned_expected_file_paths)
    false_positives = len(cleaned_unique_file_paths - cleaned_expected_file_paths)
    false_negatives = len(cleaned_expected_file_paths - cleaned_unique_file_paths)

    precision = true_positives / (true_positives + false_positives) if (true_positives + false_positives) > 0 else 0
    recall = true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) > 0 else 0

    results.append({
        'Question': question,
        'Expected File Paths': cleaned_expected_file_paths,
        'Predicted File Paths with Scores': unique_file_paths_with_scores,
        'True Positives': true_positives,
        'False Positives': false_positives,
        'False Negatives': false_negatives,
        'Precision': precision,
        'Recall': recall,
        'Retrieval Time (s)': retrieval_time,
        'Rerank Time (s)': 0
    })

results_df = pd.DataFrame(results)

total_true_positives = results_df['True Positives'].sum()
total_false_positives = results_df['False Positives'].sum()
total_false_negatives = results_df['False Negatives'].sum()
total_predictions = total_true_positives + total_false_positives

accuracy = total_true_positives / total_predictions if total_predictions != 0 else 0
average_precision = results_df['Precision'].mean()
average_recall = results_df['Recall'].mean()

print("-------------------------------------------------------------------")
print(f"Total True Positives: {total_true_positives}")
print(f"Total False Positives: {total_false_positives}")
print(f"Accuracy: {accuracy:.2f}")
print(f"Average Precision: {average_precision:.2f}")
print(f"Average Recall: {average_recall:.2f}")
print("-------------------------------------------------------------------")

# Display function for Markdown text
def display_markdown(text):
    display(Markdown(markdown.markdown(text)))

for index, row in results_df.iterrows():
    print(row['Question'])
    display_markdown(f"# Вопрос: {row['Question']}")
    display_markdown(f"Если есть llm: {qwe[index]}")
    display_markdown(f"## **Нужные PDF:**")
    for path in row['Expected File Paths']:
        display_markdown(f" - **{path}**")
    print()
    display_markdown("## **Предсказанные PDF:**")
    for path, score, text_fragment in row['Predicted File Paths with Scores']:
        display_markdown(f"  - **{path} (Score: {score:.2f})**")
        #display_markdown(f"**Фрагмент текста:**")
        #print(f"{text_fragment}")
        print()


-------------------------------------------------------------------
Total True Positives: 79
Total False Positives: 246
Accuracy: 0.24
Average Precision: 0.31
Average Recall: 0.55
-------------------------------------------------------------------
New!СБ ЗОКИИ: какие ограничения существуют по допуску иностранного гражданина к эксплуатации или обслуживанию значимого объекта КИИ?


<h1>Вопрос: New!СБ ЗОКИИ: какие ограничения существуют по допуску иностранного гражданина к эксплуатации или обслуживанию значимого объекта КИИ?</h1>

<p>Если есть llm: New!СБ ЗОКИИ: какие ограничения существуют по допуску иностранного гражданина к эксплуатации или обслуживанию значимого объекта КИИ?</p>

<h2><strong>Нужные PDF:</strong></h2>

<ul>
<li><strong>18</strong></li>
</ul>

...

<h2><strong>Предсказанные PDF:</strong></h2>

<ul>
<li><strong>25 (Score: 592.26)</strong></li>
</ul>

<ul>
<li><strong>41 (Score: 304.86)</strong></li>
</ul>

<ul>
<li><strong>1 (Score: 298.57)</strong></li>
</ul>

<ul>
<li><strong>22 (Score: 295.14)</strong></li>
</ul>